In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hw1parquets/trades.spot.parquet
/kaggle/input/hw1parquets/ticker.spot.parquet
/kaggle/input/hw1parquets/book.parquet
/kaggle/input/hw1parquets/book.csv
/kaggle/input/hw1parquets/book.spot.parquet
/kaggle/input/hw1parquets/target.parquet
/kaggle/input/hw1parquets/trades.parquet


In [2]:
import pandas as pd
import numpy as np
# РЕЖЕТ ФЛОТЫ
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
import pandas as pd
import numpy as np
book = pd.read_parquet('/kaggle/input/hw1parquets/book.parquet')

In [4]:
book.head()

,exchange,symbol,timestamp,local_timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,...,bids[22].price,bids[22].amount,asks[23].price,asks[23].amount,bids[23].price,bids[23].amount,asks[24].price,asks[24].amount,bids[24].price,bids[24].amount
0,1,1,1679443200138000000,1679443200485131264,28091.199219,16.183001,28091.099609,0.001,28091.300781,7.874,...,28087.800781,0.006,28093.599609,0.052,28087.699219,0.036,28093.699219,0.001,28087.599609,0.002
1,1,1,1679443202062000000,1679443202065465600,28090.900391,6.995000,28090.800781,0.002,28091.000000,0.735,...,28087.599609,0.002,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001
2,1,1,1679443202688000000,1679443202691394048,28090.900391,7.005000,28090.800781,0.002,28091.000000,0.735,...,28087.599609,0.002,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001
3,1,1,1679443202809000000,1679443202812150528,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,...,28087.599609,0.002,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001
4,1,1,1679443203186000000,1679443203188767744,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,...,28087.599609,0.002,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001


In [5]:
# book = import_data("/kaggle/input/hw1parquets/book.csv").sort_values('local_timestamp')
# import pandas as pd
# import numpy as np

# book = pd.read_csv('/kaggle/input/hw1parquets/book.csv')
# book['symbol'] = book['symbol'].apply(lambda x: 1 if x=='BTCUSDT' else 0)
# book['exchange'] = book['exchange'].apply(lambda x: 1 if x=='binance-futures' else 0)

# book.head(10)

In [6]:
import gc
gc.collect()
# del book

46

In [7]:
book_spot = pd.read_parquet('/kaggle/input/hw1parquets/book.spot.parquet')
book_spot['symbol'] = book_spot['symbol'].apply(lambda x: 1 if x=='BTCUSDT' else 0)
book_spot['exchange'] = book_spot['exchange'].apply(lambda x: 1 if x=='binance-futures' else 0)
book_spot.head()

,exchange,symbol,timestamp,local_timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,...,bids[22].price,bids[22].amount,asks[23].price,asks[23].amount,bids[23].price,bids[23].amount,asks[24].price,asks[24].amount,bids[24].price,bids[24].amount
0,0,1,1679443202668352000,1679443202668375808,28108.44,0.00127,28107.43,0.00071,28108.90,0.00800,...,28103.53,0.00632,28110.70,0.00036,28103.39,0.00043,28110.81,0.01614,28103.28,0.02000
1,0,1,1679443202763000000,1679443202769543168,28107.85,0.02000,28107.43,0.00036,28107.86,0.01265,...,28103.39,0.00043,28110.49,0.01599,28103.20,0.00070,28110.52,0.00660,28103.13,0.00916
2,0,1,1679443202863000000,1679443202869800960,28107.70,0.02000,28107.43,0.00159,28107.71,0.00070,...,28103.59,0.01224,28110.14,0.08048,28103.56,0.05472,28110.49,0.01599,28103.53,0.00632
3,0,1,1679443202963000000,1679443202969663232,28106.76,0.00109,28106.48,0.00070,28107.39,0.05320,...,28103.60,0.03202,28109.97,0.00065,28103.59,0.01224,28110.00,0.00200,28103.53,0.00632
4,0,1,1679443203063000000,1679443203069908224,28107.36,0.00177,28106.48,0.00124,28107.38,0.00108,...,28103.61,0.00070,28109.97,0.00065,28103.60,0.03202,28110.00,0.00200,28103.59,0.01224


In [8]:
ask_vols = ['asks[' + str(i) + '].amount' for i in range(25)]
bid_vols = ['bids[' + str(i) + '].amount' for i in range(25)]

ask_prices = ['asks[' + str(i) + '].price' for i in range(25)]
bid_prices = ['bids[' + str(i) + '].price' for i in range(25)]

prices = ask_vols+bid_vols+ask_prices+bid_prices

In [9]:
# book[prices] = book[prices].apply(pd.to_numeric, downcast='float')
# book[prices] = book[prices].apply(pd.to_numeric, downcast='integer')
end_mem = book.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
book.head()

Memory usage after optimization is: 2638.16 MB


,exchange,symbol,timestamp,local_timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,...,bids[22].price,bids[22].amount,asks[23].price,asks[23].amount,bids[23].price,bids[23].amount,asks[24].price,asks[24].amount,bids[24].price,bids[24].amount
0,1,1,1679443200138000000,1679443200485131264,28091.199219,16.183001,28091.099609,0.001,28091.300781,7.874,...,28087.800781,0.006,28093.599609,0.052,28087.699219,0.036,28093.699219,0.001,28087.599609,0.002
1,1,1,1679443202062000000,1679443202065465600,28090.900391,6.995000,28090.800781,0.002,28091.000000,0.735,...,28087.599609,0.002,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001
2,1,1,1679443202688000000,1679443202691394048,28090.900391,7.005000,28090.800781,0.002,28091.000000,0.735,...,28087.599609,0.002,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001
3,1,1,1679443202809000000,1679443202812150528,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,...,28087.599609,0.002,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001
4,1,1,1679443203186000000,1679443203188767744,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,...,28087.599609,0.002,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001


In [10]:
print(book[ask_prices].std(axis=1).median(), book[ask_prices].std(axis=1).mean(), book[ask_prices].std(axis=1).min(), book[ask_prices].std(axis=1).max())

0.93452656 1.0276445 0.6932889 38.94216


In [11]:
end_mem = book.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))

Memory usage after optimization is: 2638.16 MB


**Orderbook imbalances**

In [12]:
ask_top5 = ['asks[0].amount', 'asks[1].amount', 'asks[2].amount', 'asks[3].amount', 'asks[4].amount']
bid_top5 = ['bids[0].amount', 'bids[1].amount', 'bids[2].amount', 'bids[3].amount', 'bids[4].amount']

bid_median = book[bid_top5].stack().median()
ask_median = book[ask_top5].stack().median()

liquidity = (bid_median + ask_median) / 2

print(liquidity)
# 0.1395263671875

0.1394999921321869


In [13]:
ask_top5 = ['asks[0].amount', 'asks[1].amount', 'asks[2].amount', 'asks[3].amount', 'asks[4].amount']
bid_top5 = ['bids[0].amount', 'bids[1].amount', 'bids[2].amount', 'bids[3].amount', 'bids[4].amount']

bid_median_spot = book_spot[bid_top5].stack().median()
ask_median_spot = book_spot[ask_top5].stack().median()

liquidity_spot = (bid_median_spot + ask_median_spot) / 2

print(liquidity_spot)

0.06861


In [14]:
ask_vols = ['asks[' + str(i) + '].amount' for i in range(25)]
bid_vols = ['bids[' + str(i) + '].amount' for i in range(25)]

ask_prices = ['asks[' + str(i) + '].price' for i in range(25)]
bid_prices = ['bids[' + str(i) + '].price' for i in range(25)]

In [15]:
def calculate_average_price(volume, prices, volumes):
    cum_volumes = np.cumsum(volumes)
    total_volume = np.sum(volumes)
    if volume > total_volume:
        return np.dot(prices,volumes) / total_volume
    else:
        idx = np.where(cum_volumes > volume)[0][0]
        volumes[idx] = volume - cum_volumes[idx-1] if idx > 0 else volume
        return np.dot(prices[:idx+1],volumes[:idx+1]) / volume


In [16]:
%%time
avg_prices = np.apply_along_axis(
    lambda x:  calculate_average_price(
        liquidity, x[::2], x[1::2]), axis=1, arr=np.hstack(
        (book[ask_prices].to_numpy(), book[ask_vols].to_numpy()
        )
    )
)
# print(avg_prices[:10])
print(avg_prices.mean(), avg_prices.min(), avg_prices.max(), avg_prices.std(), np.median(avg_prices))

27899.02417926881 26590.3002788173 28879.39966027487 411.1337526771341 28036.199147888532
CPU times: user 2min 26s, sys: 1.35 s, total: 2min 27s
Wall time: 2min 27s


In [17]:
%%time
avg_prices_spot = np.apply_along_axis(
    lambda x:  calculate_average_price(
        liquidity, x[::2], x[1::2]), axis=1, arr=np.hstack(
        (book_spot[ask_prices].to_numpy(), book_spot[ask_vols].to_numpy()
        )
    )
)
# print(avg_prices[:10])
print(avg_prices_spot.mean(), avg_prices_spot.min(), avg_prices_spot.max(), avg_prices_spot.std(), np.median(avg_prices_spot))

27913.702613157442 26604.39 28868.29 420.2023565078802 28062.37
CPU times: user 43.6 s, sys: 531 ms, total: 44.1 s
Wall time: 44 s


In [18]:
book['ask_imbalance'] = avg_prices * 10000

In [19]:
book_spot['ask_imbalance'] = avg_prices_spot * 10000

In [20]:
%%time
del avg_prices
avg_prices = np.apply_along_axis(
    lambda x:  calculate_average_price(
        liquidity, x[::2], x[1::2]), axis=1, arr=np.hstack(
        (book[bid_prices].to_numpy(), book[bid_vols].to_numpy()
        )
    )
)
# print(avg_prices[:10])
print(avg_prices.mean(), avg_prices.min(), avg_prices.max(), avg_prices.std(), np.median(avg_prices))

27898.905831031487 26589.99925953509 28878.19908337024 411.1318715161577 28035.999635108463
CPU times: user 2min 24s, sys: 1.34 s, total: 2min 26s
Wall time: 2min 25s


In [21]:
%%time
del avg_prices_spot
avg_prices_spot = np.apply_along_axis(
    lambda x:  calculate_average_price(
        liquidity, x[::2], x[1::2]), axis=1, arr=np.hstack(
        (book_spot[bid_prices].to_numpy(), book_spot[bid_vols].to_numpy()
        )
    )
)
# print(avg_prices[:10])
print(avg_prices_spot.mean(), avg_prices_spot.min(), avg_prices_spot.max(), avg_prices_spot.std(), np.median(avg_prices_spot))

27913.676793128652 26604.38 28868.04 420.2002433380432 28062.35
CPU times: user 43.9 s, sys: 1.12 s, total: 45 s
Wall time: 45 s


In [22]:
book['bid_imbalance'] = avg_prices * 10000

In [23]:
book_spot['bid_imbalance'] = avg_prices_spot * 10000

In [24]:
# np.save("ask_imb.npy", book['bid_imbalance'].to_numpy())
# np.save("bid_imb.npy", book['bid_imbalance'].to_numpy())
# from IPython.display import FileLink
# FileLink('/kaggle/working/book.parquet')

Наконец-то посчитали **orderbook imbalance**. 

In [25]:
book.head()

,exchange,symbol,timestamp,local_timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,...,asks[23].price,asks[23].amount,bids[23].price,bids[23].amount,asks[24].price,asks[24].amount,bids[24].price,bids[24].amount,ask_imbalance,bid_imbalance
0,1,1,1679443200138000000,1679443200485131264,28091.199219,16.183001,28091.099609,0.001,28091.300781,7.874,...,28093.599609,0.052,28087.699219,0.036,28093.699219,0.001,28087.599609,0.002,2.809120e+08,2.809110e+08
1,1,1,1679443202062000000,1679443202065465600,28090.900391,6.995000,28090.800781,0.002,28091.000000,0.735,...,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001,2.809090e+08,2.809080e+08
2,1,1,1679443202688000000,1679443202691394048,28090.900391,7.005000,28090.800781,0.002,28091.000000,0.735,...,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001,2.809090e+08,2.809080e+08
3,1,1,1679443202809000000,1679443202812150528,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,...,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001,2.809090e+08,2.809080e+08
4,1,1,1679443203186000000,1679443203188767744,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,...,28093.400391,0.674,28087.500000,0.099,28093.500000,0.401,28087.400391,0.001,2.809090e+08,2.809080e+08


In [26]:
book_spot.head()

,exchange,symbol,timestamp,local_timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,...,asks[23].price,asks[23].amount,bids[23].price,bids[23].amount,asks[24].price,asks[24].amount,bids[24].price,bids[24].amount,ask_imbalance,bid_imbalance
0,0,1,1679443202668352000,1679443202668375808,28108.44,0.00127,28107.43,0.00071,28108.90,0.00800,...,28110.70,0.00036,28103.39,0.00043,28110.81,0.01614,28103.28,0.02000,281084400.0,281074300.0
1,0,1,1679443202763000000,1679443202769543168,28107.85,0.02000,28107.43,0.00036,28107.86,0.01265,...,28110.49,0.01599,28103.20,0.00070,28110.52,0.00660,28103.13,0.00916,281078500.0,281074300.0
2,0,1,1679443202863000000,1679443202869800960,28107.70,0.02000,28107.43,0.00159,28107.71,0.00070,...,28110.14,0.08048,28103.56,0.05472,28110.49,0.01599,28103.53,0.00632,281077000.0,281074300.0
3,0,1,1679443202963000000,1679443202969663232,28106.76,0.00109,28106.48,0.00070,28107.39,0.05320,...,28109.97,0.00065,28103.59,0.01224,28110.00,0.00200,28103.53,0.00632,281067600.0,281064800.0
4,0,1,1679443203063000000,1679443203069908224,28107.36,0.00177,28106.48,0.00124,28107.38,0.00108,...,28109.97,0.00065,28103.60,0.03202,28110.00,0.00200,28103.59,0.01224,281073600.0,281064800.0


In [27]:
book.columns.values

array(['exchange', 'symbol', 'timestamp', 'local_timestamp',
       'asks[0].price', 'asks[0].amount', 'bids[0].price',
       'bids[0].amount', 'asks[1].price', 'asks[1].amount',
       'bids[1].price', 'bids[1].amount', 'asks[2].price',
       'asks[2].amount', 'bids[2].price', 'bids[2].amount',
       'asks[3].price', 'asks[3].amount', 'bids[3].price',
       'bids[3].amount', 'asks[4].price', 'asks[4].amount',
       'bids[4].price', 'bids[4].amount', 'asks[5].price',
       'asks[5].amount', 'bids[5].price', 'bids[5].amount',
       'asks[6].price', 'asks[6].amount', 'bids[6].price',
       'bids[6].amount', 'asks[7].price', 'asks[7].amount',
       'bids[7].price', 'bids[7].amount', 'asks[8].price',
       'asks[8].amount', 'bids[8].price', 'bids[8].amount',
       'asks[9].price', 'asks[9].amount', 'bids[9].price',
       'bids[9].amount', 'asks[10].price', 'asks[10].amount',
       'bids[10].price', 'bids[10].amount', 'asks[11].price',
       'asks[11].amount', 'bids[11].price'

In [28]:
book.sort_values('timestamp', inplace=True)

In [29]:
book_spot.sort_values('timestamp', inplace=True)
# book.sort_values('timestamp', inplace=True)

In [30]:
len(book_spot)

2305858

In [31]:
len(book)

6403491

In [32]:
print(book['timestamp'].min(), book['timestamp'].max())

1679443200138000000 1679702399991000000


In [33]:
print(book_spot['timestamp'].min(), book_spot['timestamp'].max())

1679443202668352000 1679702399900000000


In [34]:
import gc
gc.collect()
# del book

230

In [35]:
# book = pd.merge_asof(book, book_spot, on='timestamp')
# print(len(book))
# book.head()

In [38]:
# book.head()

In [39]:
# both_obs.to_parquet('ob_features.parquet')

In [40]:
# del book

**Trade Imbalance**

In [41]:
horizons = [100, 250, 500, 1000, 2000]

In [42]:
book["local_timestamp"] = pd.to_datetime(book["local_timestamp"])
book.set_index('local_timestamp', inplace=True)
book.sort_index(inplace=True)
book['askVolSum'] = book[ask_vols].sum(axis=1)
book['bidVolSum'] = book[bid_vols].sum(axis=1)

In [43]:
book_spot["local_timestamp"] = pd.to_datetime(book_spot["local_timestamp"])
book_spot.set_index('local_timestamp', inplace=True)
book_spot.sort_index(inplace=True)
book_spot['askVolSum'] = book_spot[ask_vols].sum(axis=1)
book_spot['bidVolSum'] = book_spot[bid_vols].sum(axis=1)

In [44]:
chunk_size = 10000
for h in horizons:
    print(h)
    book[f'{h}msTradeImb'] = 0
    for i in range(0, len(book), chunk_size):
        chunk = book.iloc[i:i+chunk_size]
        rolling_chunk = chunk[['askVolSum','bidVolSum']].rolling(f'{h}ms', min_periods=1).sum().diff(axis=1)['bidVolSum']
        book[f'{h}msTradeImb'].iloc[i:i+chunk_size] = rolling_chunk    

100


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


250
500
1000
2000


In [45]:
chunk_size = 10000
for h in horizons:
    print(h)
    book_spot[f'{h}msTradeImb'] = 0
    for i in range(0, len(book_spot), chunk_size):
        chunk = book_spot.iloc[i:i+chunk_size]
        rolling_chunk = chunk[['askVolSum','bidVolSum']].rolling(f'{h}ms', min_periods=1).sum().diff(axis=1)['bidVolSum']
        book_spot[f'{h}msTradeImb'].iloc[i:i+chunk_size] = rolling_chunk    

100
250
500
1000
2000


In [46]:
book.head()

,exchange,symbol,timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,bids[1].price,...,bids[24].amount,ask_imbalance,bid_imbalance,askVolSum,bidVolSum,100msTradeImb,250msTradeImb,500msTradeImb,1000msTradeImb,2000msTradeImb
local_timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-22 00:00:00.485131264,1,1,1679443200138000000,28091.199219,16.183001,28091.099609,0.001,28091.300781,7.874,28091.000000,...,0.002,2.809120e+08,2.809110e+08,35.605999,0.520,-35.085999,-35.085999,-35.085999,-35.085999,-35.085999
2023-03-22 00:00:02.065465600,1,1,1679443202062000000,28090.900391,6.995000,28090.800781,0.002,28091.000000,0.735,28090.699219,...,0.001,2.809090e+08,2.809080e+08,42.733002,0.616,-42.117002,-42.117002,-42.117002,-42.117002,-77.203001
2023-03-22 00:00:02.691394048,1,1,1679443202688000000,28090.900391,7.005000,28090.800781,0.002,28091.000000,0.735,28090.699219,...,0.001,2.809090e+08,2.809080e+08,42.743004,0.616,-42.127004,-42.127004,-42.127004,-84.244006,-84.244006
2023-03-22 00:00:02.812150528,1,1,1679443202809000000,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,28090.699219,...,0.001,2.809090e+08,2.809080e+08,42.623001,0.616,-42.007001,-84.134005,-84.134005,-126.251007,-126.251007
2023-03-22 00:00:03.188767744,1,1,1679443203186000000,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,28090.699219,...,0.001,2.809090e+08,2.809080e+08,42.731998,0.616,-42.115999,-42.115999,-126.250004,-126.250004,-168.367005


In [47]:
(book['500msTradeImb'] - book['1000msTradeImb']).mean()

6.454275068907418

In [48]:
(book['1000msTradeImb'] - book['2000msTradeImb']).mean()

13.049992458090866

Супер, посчитали trade imbalance

In [50]:
book_spot.sort_values('timestamp', inplace=True)

In [51]:
book.sort_values('timestamp', inplace=True)

In [52]:
book = pd.merge_asof(book, book_spot, on='timestamp')

In [53]:
del book_spot
gc.collect()

239

In [54]:

print(len(book))
book.head()
book.to_parquet('ob_features.parquet')

6403491


In [56]:
end_mem = book.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))

Memory usage after optimization is: 8451.87 MB


In [57]:
gc.collect()

17177

In [59]:
fts = book.columns.values[3:]

In [ ]:
book[fts] = book[fts].apply(pd.to_numeric, downcast='float')
book[fts] = book[fts].apply(pd.to_numeric, downcast='integer')

In [55]:
book.columns.values

array(['exchange_x', 'symbol_x', 'timestamp', 'asks[0].price_x',
       'asks[0].amount_x', 'bids[0].price_x', 'bids[0].amount_x',
       'asks[1].price_x', 'asks[1].amount_x', 'bids[1].price_x',
       'bids[1].amount_x', 'asks[2].price_x', 'asks[2].amount_x',
       'bids[2].price_x', 'bids[2].amount_x', 'asks[3].price_x',
       'asks[3].amount_x', 'bids[3].price_x', 'bids[3].amount_x',
       'asks[4].price_x', 'asks[4].amount_x', 'bids[4].price_x',
       'bids[4].amount_x', 'asks[5].price_x', 'asks[5].amount_x',
       'bids[5].price_x', 'bids[5].amount_x', 'asks[6].price_x',
       'asks[6].amount_x', 'bids[6].price_x', 'bids[6].amount_x',
       'asks[7].price_x', 'asks[7].amount_x', 'bids[7].price_x',
       'bids[7].amount_x', 'asks[8].price_x', 'asks[8].amount_x',
       'bids[8].price_x', 'bids[8].amount_x', 'asks[9].price_x',
       'asks[9].amount_x', 'bids[9].price_x', 'bids[9].amount_x',
       'asks[10].price_x', 'asks[10].amount_x', 'bids[10].price_x',
       'bids[10

In [ ]:

# for h in horizons:
#     print(h)
#     book[f'{h}msTradeImb'] = book[['askVolSum','bidVolSum']].rolling(f'{h}ms', min_periods=1).sum().diff(axis=1)['bidVolSum']
    
# book.head()

In [ ]:
# sample = book.iloc[:10000]
# sample["local_timestamp"] = pd.to_datetime(sample["local_timestamp"])
# sample['askVolSum'] = sample[ask_vols].sum(axis=1)
# sample['bidVolSum'] = sample[bid_vols].sum(axis=1)

# sample.set_index('local_timestamp')[['askVolSum','bidVolSum']].rolling('250ms', min_periods=1).sum().diff(axis=1)['bidVolSum']

**Past returns**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# import numpy as np
# book_index = book.index                                   # 1679443200485131264
# book_index = book_index.to_frame()['local_timestamp'].astype(np.int64) # 1679443205368000000
# book_index.reset_index(drop=True)

In [21]:
trades = pd.read_parquet('/kaggle/input/hw1parquets/trades.parquet')

In [ ]:
trades.head()

In [22]:
tradesSpot = pd.read_parquet('/kaggle/input/hw1parquets/trades.spot.parquet')

In [ ]:
tradesSpot.head()

In [ ]:
# tradesSpot.drop(columns=['seq', 'remote_ts', 'remote_ts2'], inplace=True)
end_mem = tradesSpot.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
tradesSpot.head()

In [ ]:
# trades.drop(columns=['seq', 'remote_ts', 'remote_ts2'], inplace=True)
end_mem = trades.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
trades.head()

In [23]:
tradesSpot["local_timestamp_dt"] = pd.to_datetime(tradesSpot["local_ts"])
trades["local_timestamp_dt"] = pd.to_datetime(trades["local_ts"])

In [ ]:
# import numpy as np
# sample = trades.iloc[:2000]
# sample['volPriceProduct'] = sample['price'] * sample['amount']
# a = sample.rolling('50ms', min_periods=1, on='local_timestamp_dt')[['volPriceProduct', 'amount']].sum()[['volPriceProduct', 'amount']]
# a['pastReturns'] = a['volPriceProduct'] / a['amount']
# a['price'] = sample['price']
# a['local_timestamp_dt'] = sample['local_timestamp_dt']
# dif = a['price'] - a['pastReturns']

# print(dif.min(), dif.max(), dif.mean(), np.median(dif), dif.std())

In [24]:
horizons = [100, 250, 500, 1000, 2000]

In [25]:
# Считаем для окна в 50мс взвешенную цену на этом временном промежутке: 
# (\frac{1}{A} \Sigma_{(a,p)\in T^i_t} a * p)
d = 50

trades['volPriceProduct'] = trades['price'] * trades['amount']
tradesSpot['volPriceProduct'] = tradesSpot['price'] * tradesSpot['amount']

a = trades.rolling(f'{d}ms', min_periods=1, on='local_timestamp_dt')[['volPriceProduct', 'amount']].sum()[['volPriceProduct', 'amount']]
a2 = tradesSpot.rolling(f'{d}ms', min_periods=1, on='local_timestamp_dt')[['volPriceProduct', 'amount']].sum()[['volPriceProduct', 'amount']]

trades[f'weightedPrice{d}ms'] = a['volPriceProduct'] / a['amount']
tradesSpot[f'weightedPrice{d}ms'] = a2['volPriceProduct'] / a2['amount']
trades.head()

,local_ts,seq,remote_ts,remote_ts2,price,amount,side,local_timestamp_dt,volPriceProduct,weightedPrice50ms
0,1679443200000453376,3467110982,1679443199997,1679443199993,28091.1,0.001,S,2023-03-22 00:00:00.000453376,28.0911,28091.1
1,1679443200001720832,3467110983,1679443199997,1679443199993,28091.1,0.003,S,2023-03-22 00:00:00.001720832,84.2733,28091.1
2,1679443200022432256,3467110984,1679443200001,1679443199996,28091.1,0.007,S,2023-03-22 00:00:00.022432256,196.6377,28091.1
3,1679443200022432406,3467110985,1679443200001,1679443199996,28091.1,0.001,S,2023-03-22 00:00:00.022432406,28.0911,28091.1
4,1679443200022597888,3467110986,1679443200001,1679443199996,28091.1,0.004,S,2023-03-22 00:00:00.022597888,112.3644,28091.1


In [26]:
a = ((tradesSpot['price'] / tradesSpot['weightedPrice50ms']) - 1) * 10000
print(a.min(), a.max(), a.mean(), a.std(), np.median(a))

-49.217369683455246 29.241711726482045 -0.016985117374935806 0.8369165751588611 2.9527491562930663e-08


In [27]:
b = ((trades['price'] / trades['weightedPrice50ms']) - 1) * 10000 
print(b.min(), b.max(), b.mean(), b.std(), np.median(b))

-48.17427163491317 44.424332172383615 -0.0035482735249900864 1.145361440023033 8.342215807033426e-09


In [ ]:
plt.plot(b)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(a)

In [ ]:
tradesSpot.head()

In [29]:
# проверяем что сделали правильно (должно быть различие между просто колонкой price)
di = trades[f'weightedPrice{d}ms'] - trades['price']
print(di.mean(), di.std(), di.min(), di.max(), np.median(di))

0.009592759988886417 3.2052612139118843 -127.10288819113339 134.8583737405861 -2.325396053493023e-08


In [28]:
def find_window_indices(df, timestamp_col, delta):
    timestamps = df[timestamp_col].values
    timestamps_ns = timestamps.astype(np.int64)

    window_size_ns = delta * 10**6
    window_indices = np.empty(len(timestamps), dtype=np.int64)

    for i, ts in enumerate(timestamps_ns):
        lower_bound = ts - window_size_ns
        window_indices[i] = np.searchsorted(timestamps_ns, lower_bound)

    return window_indices

In [ ]:
%%time
inds = find_window_indices(trades, 'local_ts', 100)

In [ ]:
%%time
inds = find_window_indices(tradesSpot, 'local_ts', 100)

In [ ]:
%%time
for i in range(0, len(trades), chunk_size):
    inds = find_window_indices(trades.iloc[i:i+chunk_size], 'local_ts', 100)

In [ ]:
len(trades) / 1000000 * 3.16 / 60

In [30]:
%%time
# Для каждого окна из horizons посчитаем (p_t / p_{t-d} - 1) * 10к
horizons = [100, 250, 500, 1000, 2000]
chunk_size=1000000
for h in horizons:
    trades[f'PastRet{h}ms'] = 0
    for i in range(0, len(trades), chunk_size):
        chunk = trades.iloc[i:i+chunk_size][[f'weightedPrice{d}ms', 'local_ts']]
        inds = find_window_indices(chunk, 'local_ts', h)
        new = 10000 * ((chunk['weightedPrice50ms'].values / chunk['weightedPrice50ms'].iloc[inds].values) - 1)
#         new = 10000 * ((chunk['weightedPrice50ms'].values / trades['weightedPrice50ms'].values[inds]) - 1)
        trades[f'PastRet{h}ms'].iloc[i:i+chunk_size] = new
# estimated time = 5 min

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


CPU times: user 5min 29s, sys: 1.75 s, total: 5min 31s
Wall time: 5min 31s


In [ ]:
horizons

In [31]:
%%time
# Для каждого окна из horizons посчитаем (p_t / p_{t-d} - 1) * 10к
# horizons_pr = [100, 250, 500, 1000, 2000]
chunk_size=1000000
for h in horizons:
    tradesSpot[f'PastRet{h}ms'] = 0
    for i in range(0, len(tradesSpot), chunk_size):
        chunk = tradesSpot.iloc[i:i+chunk_size][[f'weightedPrice{d}ms', 'local_ts']]
        inds = find_window_indices(chunk, 'local_ts', h)
        new = 10000 * ((chunk['weightedPrice50ms'].values / chunk['weightedPrice50ms'].iloc[inds].values) - 1)
#         new = 10000 * ((chunk['weightedPrice50ms'].values / tradesSpot['weightedPrice50ms'].values[inds]) - 1)
        tradesSpot[f'PastRet{h}ms'].iloc[i:i+chunk_size] = new
# estimated time = 45 s

CPU times: user 1min 57s, sys: 734 ms, total: 1min 58s
Wall time: 1min 58s


In [ ]:
# !!!!! есть
%time
s1 = sample.iloc[find_window_indices(sample, 'local_ts')]['volPriceProduct'].values
s2 = sample['volPriceProduct'].values

s3 = ((s2 / s1) - 1) * 10000
print(s3.mean(), s3.min(), s3.max(), np.median(s3))

**Mean divirgences between markets**

In [32]:
s_horizons = [5, 9, 19, 38, 75, 150, 300, 600]
# 1. p_t у нас есть. Считаем d(p_m1, p_m2). Считаем rolling mean за d секунд из horizons. 
# Кажется тут можно воспользоваться функцией свыше. 

In [34]:
trades.head(10)

,local_ts,seq,remote_ts,remote_ts2,price,amount,side,local_timestamp_dt,volPriceProduct,weightedPrice50ms,PastRet100ms,PastRet250ms,PastRet500ms,PastRet1000ms,PastRet2000ms
0,1679443200000453376,3467110982,1679443199997,1679443199993,28091.1,0.001,S,2023-03-22 00:00:00.000453376,28.0911,28091.100,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1679443200001720832,3467110983,1679443199997,1679443199993,28091.1,0.003,S,2023-03-22 00:00:00.001720832,84.2733,28091.100,-1.110223e-12,-1.110223e-12,-1.110223e-12,-1.110223e-12,-1.110223e-12
2,1679443200022432256,3467110984,1679443200001,1679443199996,28091.1,0.007,S,2023-03-22 00:00:00.022432256,196.6377,28091.100,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,1679443200022432406,3467110985,1679443200001,1679443199996,28091.1,0.001,S,2023-03-22 00:00:00.022432406,28.0911,28091.100,-1.110223e-12,-1.110223e-12,-1.110223e-12,-1.110223e-12,-1.110223e-12
4,1679443200022597888,3467110986,1679443200001,1679443199996,28091.1,0.004,S,2023-03-22 00:00:00.022597888,112.3644,28091.100,-1.110223e-12,-1.110223e-12,-1.110223e-12,-1.110223e-12,-1.110223e-12
5,1679443200022597923,3467110987,1679443200001,1679443199996,28091.1,0.008,S,2023-03-22 00:00:00.022597923,224.7288,28091.100,2.220446e-12,2.220446e-12,2.220446e-12,2.220446e-12,2.220446e-12
6,1679443200022731264,3467110988,1679443200001,1679443199996,28091.1,0.001,S,2023-03-22 00:00:00.022731264,28.0911,28091.100,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,1679443200022743296,3467110989,1679443200001,1679443199996,28091.1,0.179,S,2023-03-22 00:00:00.022743296,5028.3069,28091.100,2.220446e-12,2.220446e-12,2.220446e-12,2.220446e-12,2.220446e-12
8,1679443202033577984,3467110990,1679443202030,1679443201081,28091.1,0.001,S,2023-03-22 00:00:02.033577984,28.0911,28091.100,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,1679443202034111488,3467110991,1679443202030,1679443201081,28091.0,0.003,S,2023-03-22 00:00:02.034111488,84.2730,28091.025,-2.669885e-02,-2.669885e-02,-2.669885e-02,-2.669885e-02,-2.669885e-02


In [35]:
tradesSpot.head(10)

,local_ts,seq,remote_ts,remote_ts2,price,amount,side,local_timestamp_dt,volPriceProduct,weightedPrice50ms,PastRet100ms,PastRet250ms,PastRet500ms,PastRet1000ms,PastRet2000ms
0,1679443200001911808,3049132203,1679443199998,1679443199997,28106.49,0.00147,S,2023-03-22 00:00:00.001911808,41.316540,28106.490000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1679443200002407168,3049132204,1679443199998,1679443199997,28105.48,0.00161,S,2023-03-22 00:00:00.002407168,45.249823,28105.962045,-0.187841,-0.187841,-0.187841,-0.187841,-0.187841
2,1679443200002408013,3049132205,1679443199998,1679443199997,28105.47,0.00018,S,2023-03-22 00:00:00.002408013,5.058985,28105.934877,-0.197507,-0.197507,-0.197507,-0.197507,-0.197507
3,1679443200004772608,3049132206,1679443200001,1679443200001,28107.81,0.00180,B,2023-03-22 00:00:00.004772608,50.594058,28106.601917,0.039819,0.039819,0.039819,0.039819,0.039819
4,1679443200004773281,3049132207,1679443200001,1679443200001,28107.81,0.00080,B,2023-03-22 00:00:00.004773281,22.486248,28106.766843,0.098498,0.098498,0.098498,0.098498,0.098498
5,1679443200005161472,3049132208,1679443200002,1679443200001,28107.81,0.00115,B,2023-03-22 00:00:00.005161472,32.323982,28106.937974,0.159385,0.159385,0.159385,0.159385,0.159385
6,1679443200008323840,3049132209,1679443200005,1679443200004,28105.47,0.00047,S,2023-03-22 00:00:00.008323840,13.209571,28106.845735,0.126567,0.126567,0.126567,0.126567,0.126567
7,1679443200008692224,3049132210,1679443200005,1679443200004,28105.42,0.00261,S,2023-03-22 00:00:00.008692224,73.355146,28106.476938,-0.004647,-0.004647,-0.004647,-0.004647,-0.004647
8,1679443200008692327,3049132211,1679443200005,1679443200005,28105.42,0.00127,S,2023-03-22 00:00:00.008692327,35.693883,28106.358776,-0.046688,-0.046688,-0.046688,-0.046688,-0.046688
9,1679443200010479616,3049132212,1679443200006,1679443200005,28107.81,0.00025,B,2023-03-22 00:00:00.010479616,7.026953,28106.390026,-0.035570,-0.035570,-0.035570,-0.035570,-0.035570


In [33]:
features = ['remote_ts'] + ['price', 'amount'] + list(trades.columns[-7:])
features

['remote_ts',
 'price',
 'amount',
 'volPriceProduct',
 'weightedPrice50ms',
 'PastRet100ms',
 'PastRet250ms',
 'PastRet500ms',
 'PastRet1000ms',
 'PastRet2000ms']

In [38]:
book.head()

,exchange,symbol,timestamp,asks[0].price,asks[0].amount,bids[0].price,bids[0].amount,asks[1].price,asks[1].amount,bids[1].price,...,bids[24].amount,ask_imbalance,bid_imbalance,askVolSum,bidVolSum,100msTradeImb,250msTradeImb,500msTradeImb,1000msTradeImb,2000msTradeImb
local_timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-22 00:00:00.485131264,1,1,1679443200138000000,28091.199219,16.183001,28091.099609,0.001,28091.300781,7.874,28091.000000,...,0.002,2.809120e+08,2.809110e+08,35.605999,0.520,-35.085999,-35.085999,-35.085999,-35.085999,-35.085999
2023-03-22 00:00:02.065465600,1,1,1679443202062000000,28090.900391,6.995000,28090.800781,0.002,28091.000000,0.735,28090.699219,...,0.001,2.809090e+08,2.809080e+08,42.733002,0.616,-42.117002,-42.117002,-42.117002,-42.117002,-77.203001
2023-03-22 00:00:02.691394048,1,1,1679443202688000000,28090.900391,7.005000,28090.800781,0.002,28091.000000,0.735,28090.699219,...,0.001,2.809090e+08,2.809080e+08,42.743004,0.616,-42.127004,-42.127004,-42.127004,-84.244006,-84.244006
2023-03-22 00:00:02.812150528,1,1,1679443202809000000,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,28090.699219,...,0.001,2.809090e+08,2.809080e+08,42.623001,0.616,-42.007001,-84.134005,-84.134005,-126.251007,-126.251007
2023-03-22 00:00:03.188767744,1,1,1679443203186000000,28090.900391,6.885000,28090.800781,0.002,28091.000000,0.735,28090.699219,...,0.001,2.809090e+08,2.809080e+08,42.731998,0.616,-42.115999,-42.115999,-126.250004,-126.250004,-168.367005


In [39]:
newTrades = trades.groupby('remote_ts').tail(1)
print(len(newTrades))
newTrades.head()

5760693


,local_ts,seq,remote_ts,remote_ts2,price,amount,side,local_timestamp_dt,volPriceProduct,weightedPrice50ms,PastRet100ms,PastRet250ms,PastRet500ms,PastRet1000ms,PastRet2000ms
1,1679443200001720832,3467110983,1679443199997,1679443199993,28091.1,0.003,S,2023-03-22 00:00:00.001720832,84.2733,28091.100,-1.110223e-12,-1.110223e-12,-1.110223e-12,-1.110223e-12,-1.110223e-12
7,1679443200022743296,3467110989,1679443200001,1679443199996,28091.1,0.179,S,2023-03-22 00:00:00.022743296,5028.3069,28091.100,2.220446e-12,2.220446e-12,2.220446e-12,2.220446e-12,2.220446e-12
9,1679443202034111488,3467110991,1679443202030,1679443201081,28091.0,0.003,S,2023-03-22 00:00:02.034111488,84.2730,28091.025,-2.669885e-02,-2.669885e-02,-2.669885e-02,-2.669885e-02,-2.669885e-02
10,1679443202780739584,3467110992,1679443202777,1679443201569,28090.9,0.547,B,2023-03-22 00:00:02.780739584,15365.7223,28090.900,0.000000e+00,0.000000e+00,0.000000e+00,-7.119693e-02,-7.119693e-02
11,1679443205348303872,3467110993,1679443205345,1679443203170,28090.9,0.118,B,2023-03-22 00:00:05.348303872,3314.7262,28090.900,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [40]:
newTradesSpot = tradesSpot.groupby('remote_ts').tail(1)
newTradesSpot

,local_ts,seq,remote_ts,remote_ts2,price,amount,side,local_timestamp_dt,volPriceProduct,weightedPrice50ms,PastRet100ms,PastRet250ms,PastRet500ms,PastRet1000ms,PastRet2000ms
2,1679443200002408013,3049132205,1679443199998,1679443199997,28105.47,0.00018,S,2023-03-22 00:00:00.002408013,5.058985,28105.934877,-0.197507,-0.197507,-0.197507,-0.197507,-0.197507
4,1679443200004773281,3049132207,1679443200001,1679443200001,28107.81,0.00080,B,2023-03-22 00:00:00.004773281,22.486248,28106.766843,0.098498,0.098498,0.098498,0.098498,0.098498
5,1679443200005161472,3049132208,1679443200002,1679443200001,28107.81,0.00115,B,2023-03-22 00:00:00.005161472,32.323982,28106.937974,0.159385,0.159385,0.159385,0.159385,0.159385
8,1679443200008692327,3049132211,1679443200005,1679443200005,28105.42,0.00127,S,2023-03-22 00:00:00.008692327,35.693883,28106.358776,-0.046688,-0.046688,-0.046688,-0.046688,-0.046688
13,1679443200010830376,3049132216,1679443200006,1679443200006,28107.94,0.00114,B,2023-03-22 00:00:00.010830376,32.043052,28106.737571,0.088083,0.088083,0.088083,0.088083,0.088083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7257136,1679702399986654208,3056391746,1679702399981,1679702399980,27454.46,0.00067,S,2023-03-24 23:59:59.986654208,18.394488,27454.462382,-0.002527,-0.002527,0.397904,0.401553,0.401553
7257137,1679702399988365312,3056391747,1679702399983,1679702399983,27454.47,0.00078,B,2023-03-24 23:59:59.988365312,21.414487,27454.463946,-0.001958,-0.001958,0.398474,0.402123,0.402123
7257138,1679702399994036992,3056391748,1679702399989,1679702399988,27454.47,0.00326,B,2023-03-24 23:59:59.994036992,89.501572,27454.466741,-0.000940,-0.000940,0.395807,0.403141,0.403141
7257140,1679702399995894272,3056391750,1679702399990,1679702399989,27454.46,0.00202,S,2023-03-24 23:59:59.995894272,55.458009,27454.465644,-0.001339,-0.001339,0.395407,0.402742,0.402742


In [41]:
m = pd.merge_asof(newTrades, newTradesSpot,on='remote_ts').dropna()
m

,local_ts_x,seq_x,remote_ts,remote_ts2_x,price_x,amount_x,side_x,local_timestamp_dt_x,volPriceProduct_x,weightedPrice50ms_x,...,amount_y,side_y,local_timestamp_dt_y,volPriceProduct_y,weightedPrice50ms_y,PastRet100ms_y,PastRet250ms_y,PastRet500ms_y,PastRet1000ms_y,PastRet2000ms_y
1,1679443200022743296,3467110989,1679443200001,1679443199996,28091.1,0.179,S,2023-03-22 00:00:00.022743296,5028.3069,28091.100000,...,0.00080,B,2023-03-22 00:00:00.004773281,22.486248,28106.766843,9.849789e-02,0.098498,0.098498,0.098498,0.098498
2,1679443202034111488,3467110991,1679443202030,1679443201081,28091.0,0.003,S,2023-03-22 00:00:02.034111488,84.2730,28091.025000,...,0.00025,S,2023-03-22 00:00:02.013705472,7.026817,28107.270000,1.576517e-10,0.281074,0.083180,-0.273752,0.291140
3,1679443202780739584,3467110992,1679443202777,1679443201569,28090.9,0.547,B,2023-03-22 00:00:02.780739584,15365.7223,28090.900000,...,0.00178,S,2023-03-22 00:00:02.776278528,50.029428,28106.549233,-1.441389e-01,-0.121240,-0.185280,0.024633,0.387552
4,1679443205348303872,3467110993,1679443205345,1679443203170,28090.9,0.118,B,2023-03-22 00:00:05.348303872,3314.7262,28090.900000,...,0.00297,S,2023-03-22 00:00:05.299558805,83.468345,28103.886365,-4.851886e-01,-0.240672,-0.339313,-0.941296,-0.922789
5,1679443205350924288,3467110994,1679443205348,1679443203176,28090.9,0.585,B,2023-03-22 00:00:05.350924288,16433.1765,28090.900000,...,0.00297,S,2023-03-22 00:00:05.299558805,83.468345,28103.886365,-4.851886e-01,-0.240672,-0.339313,-0.941296,-0.922789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5760688,1679702399575119616,3487666433,1679702399571,1679702399567,27439.0,0.031,B,2023-03-24 23:59:59.575119616,850.6090,27438.999999,...,0.00085,B,2023-03-24 23:59:59.547260206,23.336078,27454.112492,2.704558e-01,0.270457,0.274105,0.274105,0.274099
5760689,1679702399737273600,3487666434,1679702399733,1679702399730,27438.9,0.001,S,2023-03-24 23:59:59.737273600,27.4389,27438.899980,...,0.00067,B,2023-03-24 23:59:59.595739648,18.394495,27454.210913,3.026203e-01,0.306307,0.306306,0.309955,0.309949
5760690,1679702399815981568,3487666435,1679702399812,1679702399808,27438.9,0.041,S,2023-03-24 23:59:59.815981568,1124.9949,27438.900000,...,0.00067,B,2023-03-24 23:59:59.595739648,18.394495,27454.210913,3.026203e-01,0.306307,0.306306,0.309955,0.309949
5760691,1679702399991836160,3487666436,1679702399988,1679702399984,27438.9,0.051,S,2023-03-24 23:59:59.991836160,1399.3839,27438.900000,...,0.00078,B,2023-03-24 23:59:59.988365312,21.414487,27454.463946,-1.957807e-03,-0.001958,0.398474,0.402123,0.402123


In [43]:
m['dpp'] = m['weightedPrice50ms_x'] - m['weightedPrice50ms_y']
m['dpp']

1         -15.666843
2         -16.245000
3         -15.649233
4         -12.986365
5         -12.986365
             ...    
5760688   -15.112493
5760689   -15.310933
5760690   -15.310914
5760691   -15.563946
5760692   -15.564181
Name: dpp, Length: 5760692, dtype: float64

In [47]:
m["remote_ts_dp"] = pd.to_datetime(m["remote_ts_norm"])
m.set_index('remote_ts_dp', inplace=True)
m.sort_index(inplace=True)
m.head()

,local_ts_x,seq_x,remote_ts,remote_ts2_x,price_x,amount_x,side_x,local_timestamp_dt_x,volPriceProduct_x,weightedPrice50ms_x,...,local_timestamp_dt_y,volPriceProduct_y,weightedPrice50ms_y,PastRet100ms_y,PastRet250ms_y,PastRet500ms_y,PastRet1000ms_y,PastRet2000ms_y,dpp,remote_ts_norm
remote_ts_dp,,,,,,,,,,,,,,,,,,,,,
2023-03-22 00:00:00.001,1679443200022743296,3467110989,1679443200001,1679443199996,28091.1,0.179,S,2023-03-22 00:00:00.022743296,5028.3069,28091.100,...,2023-03-22 00:00:00.004773281,22.486248,28106.766843,9.849789e-02,0.098498,0.098498,0.098498,0.098498,-15.666843,1679443200001000000
2023-03-22 00:00:02.030,1679443202034111488,3467110991,1679443202030,1679443201081,28091.0,0.003,S,2023-03-22 00:00:02.034111488,84.2730,28091.025,...,2023-03-22 00:00:02.013705472,7.026817,28107.270000,1.576517e-10,0.281074,0.083180,-0.273752,0.291140,-16.245000,1679443202030000000
2023-03-22 00:00:02.777,1679443202780739584,3467110992,1679443202777,1679443201569,28090.9,0.547,B,2023-03-22 00:00:02.780739584,15365.7223,28090.900,...,2023-03-22 00:00:02.776278528,50.029428,28106.549233,-1.441389e-01,-0.121240,-0.185280,0.024633,0.387552,-15.649233,1679443202777000000
2023-03-22 00:00:05.345,1679443205348303872,3467110993,1679443205345,1679443203170,28090.9,0.118,B,2023-03-22 00:00:05.348303872,3314.7262,28090.900,...,2023-03-22 00:00:05.299558805,83.468345,28103.886365,-4.851886e-01,-0.240672,-0.339313,-0.941296,-0.922789,-12.986365,1679443205345000000
2023-03-22 00:00:05.348,1679443205350924288,3467110994,1679443205348,1679443203176,28090.9,0.585,B,2023-03-22 00:00:05.350924288,16433.1765,28090.900,...,2023-03-22 00:00:05.299558805,83.468345,28103.886365,-4.851886e-01,-0.240672,-0.339313,-0.941296,-0.922789,-12.986365,1679443205348000000


5

In [46]:
m['remote_ts_norm'] = m['remote_ts'] * 1000000

In [48]:
%%time
for h in s_horizons:
#     inds = find_window_indices(m, 'remote_ts', h * 1000) # тк эта функция работает с милисекундами
    m[f'Mean{h}s'] = m['dpp'].rolling(f'{h}s', min_periods=1).mean()
    m[f'Div{h}S'] = m['dpp'] - m[f'Mean{h}s']
m.head()

CPU times: user 2.59 s, sys: 584 ms, total: 3.18 s
Wall time: 2.89 s


,local_ts_x,seq_x,remote_ts,remote_ts2_x,price_x,amount_x,side_x,local_timestamp_dt_x,volPriceProduct_x,weightedPrice50ms_x,...,Mean38s,Div38S,Mean75s,Div75S,Mean150s,Div150S,Mean300s,Div300S,Mean600s,Div600S
remote_ts_dp,,,,,,,,,,,,,,,,,,,,,
2023-03-22 00:00:00.001,1679443200022743296,3467110989,1679443200001,1679443199996,28091.1,0.179,S,2023-03-22 00:00:00.022743296,5028.3069,28091.100,...,-15.666843,0.000000,-15.666843,0.000000,-15.666843,0.000000,-15.666843,0.000000,-15.666843,0.000000
2023-03-22 00:00:02.030,1679443202034111488,3467110991,1679443202030,1679443201081,28091.0,0.003,S,2023-03-22 00:00:02.034111488,84.2730,28091.025,...,-15.955922,-0.289078,-15.955922,-0.289078,-15.955922,-0.289078,-15.955922,-0.289078,-15.955922,-0.289078
2023-03-22 00:00:02.777,1679443202780739584,3467110992,1679443202777,1679443201569,28090.9,0.547,B,2023-03-22 00:00:02.780739584,15365.7223,28090.900,...,-15.853692,0.204459,-15.853692,0.204459,-15.853692,0.204459,-15.853692,0.204459,-15.853692,0.204459
2023-03-22 00:00:05.345,1679443205348303872,3467110993,1679443205345,1679443203170,28090.9,0.118,B,2023-03-22 00:00:05.348303872,3314.7262,28090.900,...,-15.136860,2.150495,-15.136860,2.150495,-15.136860,2.150495,-15.136860,2.150495,-15.136860,2.150495
2023-03-22 00:00:05.348,1679443205350924288,3467110994,1679443205348,1679443203176,28090.9,0.585,B,2023-03-22 00:00:05.350924288,16433.1765,28090.900,...,-14.706761,1.720396,-14.706761,1.720396,-14.706761,1.720396,-14.706761,1.720396,-14.706761,1.720396


Фух, наконец-то посчитали все фичи из статьи. Осталось только все это склеить вместе и обучить модель.

In [53]:
# БАН
# book.reset_index(inplace=True)

In [ ]:
book.head()

In [ ]:
m.to_parquet('features.parquet', coerce_timestamps='us',
    allow_truncated_timestamps=True,)

In [50]:
import gc
gc.collect()

6764

In [ ]:
# chunk_size=1000000
# for h in s_horizons:
#     tradesSpot[f'Mean{h}S'] = 0
#     for i in range(0, len(tradesSpot), chunk_size):
#         chunk = m.iloc[i:i+chunk_size][['dpp', 'remote_ts']]
#         inds = find_window_indices(chunk, 'remote_ts', h * 1000) # тк эта функция работает с милисекундами
#         new = 10000 * ((chunk['weightedPrice50ms'].values / chunk['weightedPrice50ms'].iloc[inds].values) - 1)
# #         new = 10000 * ((chunk['weightedPrice50ms'].values / tradesSpot['weightedPrice50ms'].values[inds]) - 1)
#         tradesSpot[f'PastRet{h}ms'].iloc[i:i+chunk_size] = new

In [36]:
for h in horizons:
    print(trades[f'PastRet{h}ms'].mean(), trades[f'PastRet{h}ms'].min(), trades[f'PastRet{h}ms'].max(), trades[f'PastRet{h}ms'].std())

-0.008904632481654719 -51.80572242627912 51.12517580497355 1.5722002691972359
-0.0037393117841916378 -58.9025169834656 52.073026816326085 2.0945335633119733
0.005788656402696163 -39.50726953437456 58.93026885488206 2.667469011044787
0.01002109831783046 -40.02055505630153 57.89277463161157 3.5296860464262423
0.022566449671674108 -56.52941034151926 72.48775074710112 5.122759222877512


In [37]:
for h in horizons:
    print(tradesSpot[f'PastRet{h}ms'].mean(), tradesSpot[f'PastRet{h}ms'].min(), tradesSpot[f'PastRet{h}ms'].max(), tradesSpot[f'PastRet{h}ms'].std())

-0.0315344921628754 -35.006895566986884 33.32973395584471 1.2268194908462573
-0.046059735730426266 -32.96728938020688 49.45906069677264 1.7726565032056967
-0.05671626951754761 -26.879601983922363 48.11717344646027 2.338252659367034
-0.06861721670408252 -38.42591595034306 54.830986887446365 3.2835135219525307
-0.1144598580423071 -58.67877932034582 50.899663063506125 4.7950052759758925


In [ ]:
# допустим у нас все хорошо посчиталось сверху. осталось для изначальных p_50ms посчитать:
# дано k секунд, надо посчитать по ним rolling mean. 
dif12 = trades['weightedPrice50ms'] - tradesSpot['weightedPrice50ms']
dif['local_ts']
dif12.head()

In [49]:
target = pd.read_parquet('/kaggle/input/hw1parquets/target.parquet')
print("Target Length", len(target))
target.head()

Target Length 37565580


,local_ts,target
0,1679443200002269696,0
1,1679443200023076352,0
2,1679443200025819392,0
3,1679443200041581312,0
4,1679443200045155072,0


In [ ]:
# merging
df1 = pd.merge_asof(target,m, on='local_ts',direction='backward')
print("Result len: ", len(df1))
df1.head()

In [ ]:
del target
del ticker

In [ ]:
# # merging
# df = pd.merge_asof(df,trades, on='local_ts',direction='backward')
# print("Result len: ", len(df))
# df.head()

In [ ]:
bookmin = book['local_timestamp'].min()
len(df1['local_ts'][df1['local_ts'] < bookmin])

In [ ]:
df1 = df1[df1['local_ts'] >= bookmin]

In [ ]:
print(target['local_ts'].min(), book['local_timestamp'].min())

In [ ]:
# merging

df = pd.merge_asof(df1,book, left_on='local_ts', right_on='local_timestamp',direction='backward')
del df1
print("Result len: ", len(df))
df.head()

In [ ]:
end_mem = df.memory_usage().sum() / 1024**2
print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))

# ГРУЗИМ ДАННЫЕ 100000ЫЙ РАЗ

In [10]:
import pandas as pd
import numpy as np

In [11]:
tf = pd.read_parquet('/kaggle/input/hw1parquets/tradeFeatures.parquet')

In [ ]:
# a = pd.read_csv('final.csv')

In [12]:
import gc
tf.rename(columns={"local_ts_x": "local_ts"}, inplace=True)
good_cols = ['local_ts', 'PastRet100ms_x',
'PastRet250ms_x', 'PastRet500ms_x', 'PastRet1000ms_x', 'PastRet2000ms_x',
'PastRet250ms_y', 'PastRet500ms_y', 'PastRet1000ms_y', 'PastRet2000ms_y',
'Div5S', 'Div9S', 'Div19S', 'Div38S', 'Div75S', 'Div150S', 'Div300S', 'Div600S'
]
tf = tf[good_cols]
gc.collect()

69

In [13]:
tf.memory_usage(deep=True).sum() / 1024**2

791.1107788085938

In [14]:
float_cols = tf.select_dtypes(include='float').columns.tolist()
tf[float_cols]=tf[float_cols].astype('float16')
tf.head()

,local_ts,PastRet100ms_x,PastRet250ms_x,PastRet500ms_x,PastRet1000ms_x,PastRet2000ms_x,PastRet250ms_y,PastRet500ms_y,PastRet1000ms_y,PastRet2000ms_y,Div5S,Div9S,Div19S,Div38S,Div75S,Div150S,Div300S,Div600S
0,1679443200022743296,0.000000,0.000000,0.000000,0.000000,0.000000,0.098511,0.098511,0.098511,0.098511,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1679443202034111488,-0.026703,-0.026703,-0.026703,-0.026703,-0.026703,0.281006,0.083191,-0.273682,0.291260,-0.289062,-0.289062,-0.289062,-0.289062,-0.289062,-0.289062,-0.289062,-0.289062
2,1679443202780739584,0.000000,0.000000,0.000000,-0.071167,-0.071167,-0.121216,-0.185303,0.024628,0.387451,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468
3,1679443205348303872,0.000000,0.000000,0.000000,0.000000,0.000000,-0.240723,-0.339355,-0.941406,-0.922852,1.973633,2.150391,2.150391,2.150391,2.150391,2.150391,2.150391,2.150391
4,1679443205350924288,0.000000,0.000000,0.000000,0.000000,0.000000,-0.240723,-0.339355,-0.941406,-0.922852,1.480469,1.720703,1.720703,1.720703,1.720703,1.720703,1.720703,1.720703


In [15]:
tf.memory_usage(deep=True).sum() / 1024**2

230.74073028564453

In [7]:
# tf.drop(columns=['side_x', 'side_y'], inplace=True)

In [8]:
# tf.memory_usage().sum() / 1024**2

1186.6661071777344

In [16]:
import numpy as np
import pandas as pd
ob = pd.read_parquet('/kaggle/input/hw1parquets/ob_features.parquet')

In [17]:
ob_f = ['timestamp', '100msTradeImb_x',
       '250msTradeImb_x', '500msTradeImb_x', '1000msTradeImb_x',
       '2000msTradeImb_x',
        '100msTradeImb_y',
       '250msTradeImb_y', '500msTradeImb_y', '1000msTradeImb_y',
       '2000msTradeImb_y']
ob = ob[ob_f]
gc.collect()

46

In [17]:
# base_cols1 = [f'asks[{i}].price_x' for i in range(25)]
# base_cols2 = [f'asks[{i}].price_y' for i in range(25)]
# base_cols3 = [f'asks[{i}].amount_x' for i in range(25)]
# base_cols4 = [f'asks[{i}].amount_y' for i in range(25)]

# base_cols5 = [f'bids[{i}].price_x' for i in range(25)]
# base_cols6 = [f'bids[{i}].price_y' for i in range(25)]
# base_cols7 = [f'bids[{i}].amount_x' for i in range(25)]
# base_cols8 = [f'bids[{i}].amount_y' for i in range(25)]

# base_cols = base_cols1 + base_cols2 + base_cols3 + base_cols4 + base_cols5 + base_cols6 + base_cols7 + base_cols8   

# ob.drop(columns=base_cols, inplace=True)

In [18]:
ob.memory_usage().sum() / 1024**2

586.2571105957031

In [43]:
ob.columns

Index(['timestamp', 'ask_imbalance_x', 'bid_imbalance_x', 'askVolSum_x',
       'bidVolSum_x', '100msTradeImb_x', '250msTradeImb_x', '500msTradeImb_x',
       '1000msTradeImb_x', '2000msTradeImb_x', 'ask_imbalance_y',
       'bid_imbalance_y', 'askVolSum_y', 'bidVolSum_y', '100msTradeImb_y',
       '250msTradeImb_y', '500msTradeImb_y', '1000msTradeImb_y',
       '2000msTradeImb_y'],
      dtype='object')

In [ ]:
#1679443200002269696
#1679443200138000000

In [19]:
print(len(ob))
ob.dropna(inplace=True)
tf.dropna(inplace=True)
print(len(ob))

6403491
6403489


In [13]:
pd.to_datetime(1679443200138000000)

Timestamp('2023-03-22 00:00:00.138000')

In [14]:
pd.to_datetime(1679443200002269696)

Timestamp('2023-03-22 00:00:00.002269696')

In [20]:
target = pd.read_parquet('/kaggle/input/hw1parquets/target.parquet')
print(target.memory_usage().sum() / 1024**2)
target['target'] = target['target'].astype('int8')
print(target.memory_usage().sum() / 1024**2)
print(len(target))
target.head()

573.2053833007812
322.4280815124512
37565580


,local_ts,target
0,1679443200002269696,0
1,1679443200023076352,0
2,1679443200025819392,0
3,1679443200041581312,0
4,1679443200045155072,0


In [48]:
ob[ob.columns[1:]].dtypes

ask_imbalance_x     float64
bid_imbalance_x     float64
askVolSum_x         float32
bidVolSum_x         float32
100msTradeImb_x     float64
250msTradeImb_x     float64
500msTradeImb_x     float64
1000msTradeImb_x    float64
2000msTradeImb_x    float64
ask_imbalance_y     float64
bid_imbalance_y     float64
askVolSum_y         float64
bidVolSum_y         float64
100msTradeImb_y     float64
250msTradeImb_y     float64
500msTradeImb_y     float64
1000msTradeImb_y    float64
2000msTradeImb_y    float64
dtype: object

In [21]:
ob_float_cols = ob.select_dtypes(include='float').columns.tolist()
ob[ob_float_cols]=ob[ob_float_cols].astype('float16')
ob.head()

,timestamp,100msTradeImb_x,250msTradeImb_x,500msTradeImb_x,1000msTradeImb_x,2000msTradeImb_x,100msTradeImb_y,250msTradeImb_y,500msTradeImb_y,1000msTradeImb_y,2000msTradeImb_y
2,1679443202688000000,-42.1250,-42.1250,-42.1250,-84.250,-84.250,-1.689453,-1.689453,-1.689453,-1.689453,-1.689453
3,1679443202809000000,-42.0000,-84.1250,-84.1250,-126.250,-126.250,-0.365479,-2.054688,-2.054688,-2.054688,-2.054688
4,1679443203186000000,-42.1250,-42.1250,-126.2500,-126.250,-168.375,-0.165894,-0.556152,-1.355469,-3.044922,-3.044922
5,1679443203339000000,-41.6875,-83.8125,-83.8125,-168.000,-210.125,0.040588,-0.108032,-1.297852,-2.986328,-2.986328
6,1679443203409000000,-82.8750,-125.0000,-125.0000,-209.125,-251.250,-0.776855,-0.736328,-1.708984,-3.763672,-3.763672


In [22]:
ob.memory_usage().sum() / 1024**2

219.8463478088379

In [23]:
n = pd.merge_asof(target, ob, left_on='local_ts', right_on='timestamp')
n.dropna(inplace=True)
n.head()

,local_ts,target,timestamp,100msTradeImb_x,250msTradeImb_x,500msTradeImb_x,1000msTradeImb_x,2000msTradeImb_x,100msTradeImb_y,250msTradeImb_y,500msTradeImb_y,1000msTradeImb_y,2000msTradeImb_y
29,1679443202780739702,0,1.679443e+18,-42.12500,-42.12500,-42.1250,-84.2500,-84.250,-1.689453,-1.689453,-1.689453,-1.689453,-1.689453
30,1679443205344112384,-1,1.679443e+18,-42.15625,-42.15625,-84.1875,-84.1875,-209.125,0.299072,0.455322,0.620605,-0.309570,-4.218750
31,1679443205349040896,-1,1.679443e+18,-42.15625,-42.15625,-84.1875,-84.1875,-209.125,0.299072,0.455322,0.620605,-0.309570,-4.218750
32,1679443205350975232,-1,1.679443e+18,-42.15625,-42.15625,-84.1875,-84.1875,-209.125,0.299072,0.455322,0.620605,-0.309570,-4.218750
33,1679443205352709632,-1,1.679443e+18,-40.03125,-40.03125,-82.1875,-124.1875,-165.375,-0.193115,-0.348877,-0.192505,-0.010986,-4.906250


In [24]:
n.memory_usage().values.sum() / 1024**2

1612.1385526657104

In [58]:
del ob
del target

In [59]:
import gc 
gc.collect()

69

In [25]:
tf.head()

,local_ts,PastRet100ms_x,PastRet250ms_x,PastRet500ms_x,PastRet1000ms_x,PastRet2000ms_x,PastRet250ms_y,PastRet500ms_y,PastRet1000ms_y,PastRet2000ms_y,Div5S,Div9S,Div19S,Div38S,Div75S,Div150S,Div300S,Div600S
0,1679443200022743296,0.000000,0.000000,0.000000,0.000000,0.000000,0.098511,0.098511,0.098511,0.098511,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1679443202034111488,-0.026703,-0.026703,-0.026703,-0.026703,-0.026703,0.281006,0.083191,-0.273682,0.291260,-0.289062,-0.289062,-0.289062,-0.289062,-0.289062,-0.289062,-0.289062,-0.289062
2,1679443202780739584,0.000000,0.000000,0.000000,-0.071167,-0.071167,-0.121216,-0.185303,0.024628,0.387451,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468
3,1679443205348303872,0.000000,0.000000,0.000000,0.000000,0.000000,-0.240723,-0.339355,-0.941406,-0.922852,1.973633,2.150391,2.150391,2.150391,2.150391,2.150391,2.150391,2.150391
4,1679443205350924288,0.000000,0.000000,0.000000,0.000000,0.000000,-0.240723,-0.339355,-0.941406,-0.922852,1.480469,1.720703,1.720703,1.720703,1.720703,1.720703,1.720703,1.720703


In [61]:
tf.columns

Index(['local_ts', 'PastRet100ms_x', 'PastRet250ms_x', 'PastRet500ms_x',
       'PastRet1000ms_x', 'PastRet2000ms_x', 'PastRet250ms_y',
       'PastRet500ms_y', 'PastRet1000ms_y', 'PastRet2000ms_y', 'Div5S',
       'Div9S', 'Div19S', 'Div38S', 'Div75S', 'Div150S', 'Div300S', 'Div600S'],
      dtype='object')

In [29]:
# tf.drop(columns=['remote_ts_dp', 'seq_x', 'seq_y', 'remote_ts2_x','remote_ts2_y', 'local_timestamp_dt_x','local_timestamp_dt_y', 'local_ts_y', 'remote_ts_norm'], inplace=True)

In [30]:
# tf.drop(columns=['remote_ts'], inplace=True)

In [62]:
# tf.head()

In [26]:
import gc 
gc.collect()

1071

In [27]:
tf.memory_usage().values.sum() / 1024**2

274.69120025634766

In [28]:
tf.isna().sum()

local_ts           0
PastRet100ms_x     0
PastRet250ms_x     0
PastRet500ms_x     0
PastRet1000ms_x    0
PastRet2000ms_x    0
PastRet250ms_y     0
PastRet500ms_y     0
PastRet1000ms_y    0
PastRet2000ms_y    0
Div5S              0
Div9S              0
Div19S             0
Div38S             0
Div75S             0
Div150S            0
Div300S            0
Div600S            0
dtype: int64

In [66]:
n.isna().sum()

local_ts            0
target              0
timestamp           0
ask_imbalance_x     0
bid_imbalance_x     0
askVolSum_x         0
bidVolSum_x         0
100msTradeImb_x     0
250msTradeImb_x     0
500msTradeImb_x     0
1000msTradeImb_x    0
2000msTradeImb_x    0
ask_imbalance_y     0
bid_imbalance_y     0
askVolSum_y         0
bidVolSum_y         0
100msTradeImb_y     0
250msTradeImb_y     0
500msTradeImb_y     0
1000msTradeImb_y    0
2000msTradeImb_y    0
dtype: int64

In [29]:
final = pd.merge_asof(n, tf, on='local_ts')
final.head()

,local_ts,target,timestamp,100msTradeImb_x,250msTradeImb_x,500msTradeImb_x,1000msTradeImb_x,2000msTradeImb_x,100msTradeImb_y,250msTradeImb_y,...,PastRet1000ms_y,PastRet2000ms_y,Div5S,Div9S,Div19S,Div38S,Div75S,Div150S,Div300S,Div600S
0,1679443202780739702,0,1.679443e+18,-42.12500,-42.12500,-42.1250,-84.2500,-84.250,-1.689453,-1.689453,...,0.024628,0.387451,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468
1,1679443205344112384,-1,1.679443e+18,-42.15625,-42.15625,-84.1875,-84.1875,-209.125,0.299072,0.455322,...,0.024628,0.387451,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468,0.204468
2,1679443205349040896,-1,1.679443e+18,-42.15625,-42.15625,-84.1875,-84.1875,-209.125,0.299072,0.455322,...,-0.941406,-0.922852,1.973633,2.150391,2.150391,2.150391,2.150391,2.150391,2.150391,2.150391
3,1679443205350975232,-1,1.679443e+18,-42.15625,-42.15625,-84.1875,-84.1875,-209.125,0.299072,0.455322,...,-0.941406,-0.922852,1.480469,1.720703,1.720703,1.720703,1.720703,1.720703,1.720703,1.720703
4,1679443205352709632,-1,1.679443e+18,-40.03125,-40.03125,-82.1875,-124.1875,-165.375,-0.193115,-0.348877,...,-0.941406,-0.922852,1.480469,1.720703,1.720703,1.720703,1.720703,1.720703,1.720703,1.720703


In [31]:
del n
del tf
gc.collect()

46

In [32]:
final.memory_usage().values.sum() / 1024**2

2830.1987924575806

In [34]:
final.drop(columns=['local_ts'], inplace=True)

In [35]:
final.drop(columns=['timestamp'], inplace=True)

In [33]:
from sklearn.model_selection import train_test_split

In [36]:
y = final['target']
X = final.drop(columns=['target'])

In [37]:
del final
gc.collect()

18504

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

In [39]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [41]:
model.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [42]:
y_pred = model.predict(X_test)

In [43]:
import numpy as np
from sklearn.metrics import confusion_matrix

def metric(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    weight_matrix = np.array([
        [1.9,     0,    -2],
        [-0.3,    0,    -0.3],
        [-2,      0,     1.9]
    ])
    
    hit_matrix = conf_matrix * weight_matrix
    hit_matrix_sum = np.sum(hit_matrix)
    action_count = np.sum(conf_matrix[:, 0]) + np.sum(conf_matrix[:, 2])
    if action_count == 0:
        return 0
    else:
        return hit_matrix_sum / np.sqrt(action_count)

In [44]:
res = metric(y_pred, y_test)

In [45]:
res

452.1230859237322